In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

def matchDat(afflst, hladic, aadic):
    seqlst = []
    tablst = []
    header = []
    for affin in afflst:
        affstr = affin.strip().split('\t')
        if affstr[0] in hladic:
            hlaseq = hladic[affstr[0]]
            aaseq = affstr[1]
            tmp = []
            tmp0 = []
            for hlain in hlaseq:
                for aain in aaseq:
                    if hlain == 'X' or aain == 'X':
                        tmp0.append([0.0])
                    elif hlain == '*' or hlain == '.':
                        tmp0.append([0.0])
                    elif aain == 'U':
                        tmp0.append([aadic[hlain, 'C']])
                    elif aain == 'J':
                        aa1 = aadic[hlain, 'L']
                        aa2 = aadic[hlain, 'I']
                        aamax = max(aa1, aa2)
                        tmp0.append([aamax])
                    elif aain == 'Z':
                        aa1 = aadic[hlain, 'Q']
                        aa2 = aadic[hlain, 'E']
                        aamax = max(aa1, aa2)
                        tmp0.append([aamax])
                    elif aain == 'B':
                        aa1 = aadic[hlain, 'D']
                        aa2 = aadic[hlain, 'N']
                        aamax = max(aa1, aa2)
                        tmp0.append([aamax])
                    else:
                        tmp0.append([aadic[hlain, aain]])
                tmp.append(tmp0)
                tmp0 = []
            seqlst.append(np.array(tmp).T)
            tablst.append(int(affstr[2]))
            header.append((affstr[0], affstr[1]))
    seqarray0 = np.array(seqlst, dtype=np.float32)
    a_seq2 = seqarray0.reshape(seqarray0.shape[0], -1)
    a_lab2 = np.array(tablst, dtype=np.float32)
    return (a_seq2, a_lab2), header

def load_data(affydatin_test, seqdatin):
    hladicin = {x.strip().split('\t')[0]: x.strip().split('\t')[1] for x in open(seqdatin).readlines()}
    aalst = open('data/Calpha.txt').readlines()
    aadicin = {}
    aaseq0 = aalst[0].strip().split('\t')
    for aain in aalst[1:]:
        aastr = aain.strip().split('\t')
        for i in range(1, len(aastr)):
            aadicin[aaseq0[i - 1], aastr[0]] = float(aastr[i])
    afflst = open(affydatin_test).readlines()
    (a_seq2, a_lab2), header = matchDat(afflst, hladicin, aadicin)
    return a_seq2, a_lab2

# 데이터 파일 경로
Datname = 'data/class1_input.dat'
Seqdatin = 'data/All_prot_alignseq_C_369.dat'

# 전처리 수행
sequences, labels = load_data(Datname, Seqdatin)

In [4]:
print("Train set X shape:", train_set[0].shape)  # 예: (N, 15*2214)
print("Validation set X shape:", valid_set[0].shape)
print("Test set X shape:", test_set[0].shape)

NameError: name 'train_set' is not defined

In [5]:
sequences.shape

(202, 3321)

In [2]:
class CNN(nn.Module):
    def __init__(self, in_dim, nkerns, filtsize, poolsize, hidden):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, nkerns[0], kernel_size=filtsize[0]),
            nn.ReLU(),
            nn.MaxPool2d(poolsize[0])
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(nkerns[0], nkerns[1], kernel_size=filtsize[1]),
            nn.ReLU(),
            nn.MaxPool2d(poolsize[1])
        )
        self.fc1 = nn.Linear(nkerns[1] * (in_dim[0] - filtsize[0][0] + 1 - filtsize[1][0] + 1) // poolsize[0][0] // poolsize[1][0] *
                             (in_dim[1] - filtsize[0][1] + 1 - filtsize[1][1] + 1) // poolsize[0][1] // poolsize[1][1], hidden)
        self.fc2 = nn.Linear(hidden, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# 모델 하이퍼파라미터 설정
in_dim = (20, 20)  # 입력 데이터의 차원 (예: 20x20)
nkerns = [10, 20]  # 각 컨볼루션 레이어의 필터 수
filtsize = [(5, 5), (5, 5)]  # 각 컨볼루션 레이어의 필터 크기
poolsize = [(2, 2), (2, 2)]  # 각 풀링 레이어의 풀링 크기
hidden = 100  # 완전 연결 레이어의 노드 수

# 모델 초기화
model = CNN(in_dim, nkerns, filtsize, poolsize, hidden)

# 손실 함수 및 옵티마이저 정의
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 데이터 텐서로 변환
sequences_tensor = torch.tensor(sequences).reshape(-1, 1, in_dim[0], in_dim[1])
labels_tensor = torch.tensor(labels).reshape(-1, 1)

# 데이터셋 및 데이터로더 정의
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(sequences_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    for i, (seqs, lbls) in enumerate(dataloader):
        outputs = model(seqs)
        loss = criterion(outputs, lbls)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

RuntimeError: shape '[-1, 1, 20, 20]' is invalid for input of size 670842